In [20]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout,Flatten,Dense

In [21]:
# parameters
img_width,img_height=256,256
batch_size=10
epochs=10
train_data_dir='../data/dogs-vs-cats/train'
validation_data_dir='../data/dogs-vs-cats/validation'
OUT_CATEGORIES=1
nb_train_samples=200
nb_validation_samples=100

In [22]:
# load vgg16 model pretrained
base_model=applications.VGG16(weights='imagenet',include_top=False,
                              input_shape=(img_width,img_height,3))
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [23]:
# 冻结
for layer in base_model.layers[:15]:
    layer.trainable=False

In [24]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [25]:
base_model.output_shape[1:]

(8, 8, 512)

In [26]:
# 添加一组自定义顶层
top_model=Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(OUT_CATEGORIES,activation='sigmoid'))

In [27]:
top_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 8,389,121
Trainable params: 8,389,121
Non-trainable params: 0
_________________________________________________________________


In [28]:
# create final model
model=Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=0.0001,momentum=0.9),
             metrics=['accuracy'])

In [29]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [ ]:
# train
train_datagen=ImageDataGenerator(rescale=1.0/255, horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='binary')
validation_generator=test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    verbose=2,workers=12)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/10
 - 226s - loss: 0.7252 - accuracy: 0.5900 - val_loss: 0.8094 - val_accuracy: 0.5600
Epoch 2/10
 - 234s - loss: 0.7055 - accuracy: 0.6100 - val_loss: 0.8027 - val_accuracy: 0.6200
Epoch 3/10
 - 237s - loss: 0.6186 - accuracy: 0.6750 - val_loss: 0.4361 - val_accuracy: 0.6700
Epoch 4/10
 - 257s - loss: 0.6066 - accuracy: 0.6500 - val_loss: 0.3322 - val_accuracy: 0.6400
Epoch 5/10
 - 231s - loss: 0.5386 - accuracy: 0.7300 - val_loss: 0.3011 - val_accuracy: 0.6400
Epoch 6/10
 - 231s - loss: 0.4189 - accuracy: 0.8200 - val_loss: 0.3717 - val_accuracy: 0.8600
Epoch 7/10


In [ ]:
score=model.evaluate_generator(validation_generator, nb_validation_samples/batch_size)
scores=model.predict_generator(validation_generator, nb_validation_samples/batch_size)

- 如果新的数据集很小，并且与ImageNet数据集相似，那么可以冻结所有的VGG16网络并且仅重新训练定制网络。这样可以最小化组合网络过拟合的风险。
- 如果新数据集很大并且与ImageNet数据集相似，那么可以重新训练整个并置网络。仍然保持预先计算的权重作为训练起点。
- 如果新数据集与ImageNet数据集有很大的不同，实际上仍然可以使用预训练模型的权值进行初始化。在这种情况下，将有足够的数据和信心 通过整个网络进行微调。